1. **Word sense disambiguation** - assign the most appropriate meaning to a polysemous word within a given context  
Example:  
*I can hear bass sound.*  
*He likes to eat grilled bass.*  
  
2. **WordNet** - lexical database of semantic relations between words  
Words are grouped into synsets (sets of synonyms) that express a distinct concept  
Relations between synsets:  
hyperonymy/hyponymy (IS_A relation) - furniture, bed  
meronymy/holonymy (part of relation) - finger, hand  
  
3. **PageRank algorithm** - rank web pages  
Main idea - a lot of important web pages have links to an important web page
  
Combination of all 3:
- Create a graph from words based on their WordNet relations
- Apply PageRank algorithm to that graph
- Assign meaning to each polysemous word based on the rank 

In [15]:
import nltk
nltk.download('punkt')
nltk.download('semcor')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from nltk.corpus import wordnet as wn
from nltk.corpus import semcor
from nltk.corpus.reader.semcor import SemcorCorpusReader

from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
from nltk.corpus import stopwords

import string

import networkx as nx

import random

[nltk_data] Downloading package punkt to /home/stefan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package semcor to /home/stefan/nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/stefan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/stefan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/stefan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# example semcor file
file = semcor.fileids()[7]
reader = SemcorCorpusReader(semcor.root, semcor.fileids(), wn)
words = reader.words(file)
senses = reader.tagged_chunks(file, 'sem')
for i in range(30):
    print('word: "{}", meaning: {}'.format(words[i], senses[i]))

word: "Sizzling", meaning: (Lemma('sizzling.s.01.sizzling') Sizzling)
word: "temperatures", meaning: (Lemma('temperature.n.01.temperature') temperatures)
word: "and", meaning: ['and']
word: "hot", meaning: (Lemma('hot.a.01.hot') hot)
word: "summer", meaning: (Lemma('summer.n.01.summer') summer)
word: "pavements", meaning: (Lemma('pavement.n.01.pavement') pavements)
word: "are", meaning: (Lemma('be.v.01.be') are)
word: "anything", meaning: ['anything']
word: "but", meaning: ['but']
word: "kind", meaning: (kind.s.00 kind)
word: "to", meaning: ['to']
word: "the", meaning: ['the']
word: "feet", meaning: (Lemma('foot.n.01.foot') feet)
word: ".", meaning: ['.']
word: "That", meaning: ['That']
word: "is", meaning: ['is']
word: "why", meaning: ['why']
word: "it", meaning: ['it']
word: "is", meaning: (Lemma('be.v.01.be') is)
word: "important", meaning: (Lemma('important.a.01.important') important)
word: "to", meaning: ['to']
word: "invest", meaning: (Lemma('invest.v.01.invest') invest)
word: "i

In [17]:
NUM_OF_FILES = 5

In [18]:
# main function for word sense dismabiguation
def wsd():
    avg_acc = 0
    files = random.sample(semcor.fileids(), NUM_OF_FILES)
#     files = [semcor.fileids()[7]]
    print(files)
    for file in files:
        clean_words, original_to_lemma = preprocess(file)
        word_senses = dict([(w, None) for w in clean_words])
        
        G = create_graph(clean_words, word_senses)
        
        pr = nx.pagerank(G)
        
        avg_acc += evaluate(pr, word_senses, original_to_lemma, file)
    
    avg_acc /= NUM_OF_FILES
    print('Average wsd accuracy: {} achieved on {} files'.format(avg_acc, NUM_OF_FILES))

In [19]:
# preprocessing of the semcor file - pos tagging, lemmatization, stopwords removal, collocations identification
def preprocess(file):
    reader = SemcorCorpusReader(semcor.root, semcor.fileids(), wn)
    
    words = reader.words(file)
    
    # pos tagging
    tagged_words = nltk.pos_tag(words)
    # convert to wordnet pos tag
    tagged_words = list(map(lambda pair: (pair[0], to_wordnet_pos(pair[1][0])), tagged_words))
    # filter out those without wordnet tag
    tagged_words = list(filter(lambda pair: pair[1] != None, tagged_words))
    
    # lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(w, tag) for (w, tag) in tagged_words]
    
    # map from original word to lemmatized
    original_to_lemma = {(w.lower(), i) : lemmatizer.lemmatize(w, tag).lower() for (i, (w, tag)) in enumerate(tagged_words)}

    
    
    # stopwords
    english_stopwords = stopwords.words('english')
    punctuation = string.punctuation
    normalized_words = map(lambda x: x.lower(), lemmatized_words)
    clean_words = list(filter(lambda x: x not in english_stopwords and x not in punctuation, normalized_words))
    
    original_to_lemma = {k : v for k, v in original_to_lemma.items() if v not in english_stopwords and v not in punctuation}


    # collocations
    clean_words, original_to_lemma = wn_collocations(clean_words, original_to_lemma)

    
    #remove duplicates - order doesn't matter to pagerank
    clean_words = list(set(clean_words))
    
    return clean_words, original_to_lemma

In [20]:
# convert nltk pos tag to wordnet's pos tag
def to_wordnet_pos(nltk_pos):
    if nltk_pos == "J":
        return wn.ADJ
    elif nltk_pos == "N":
        return wn.NOUN
    elif nltk_pos == "V":
        return wn.VERB
    elif nltk_pos == "R":
        return wn.ADV
    elif nltk_pos == "S":
        return wn.ADJ_SAT
    else:
        return None

In [21]:
# find collocations - sequence of words that forms a compound concept defined in wordnet
def wn_collocations(words, original_to_lemma):
    # for every col_size group of adjacent words check if they are a collocation
    # i.e. there is a meaning in wordnet for the group
    max_col_size = 5
    for col_size in range(max_col_size, 1, -1):
        to_delete = []
        to_delete_o2l = []
        cols_to_add = []
        cols_to_add_ind = []
        for (k, ix), v in original_to_lemma.items():
            if ix > len(words) - col_size:
                break
            if ix in to_delete:
                continue
            clean_col = "_".join([words[j] for j in range(ix, ix + col_size)])
            orig_col = "_".join([k for (k, index) in original_to_lemma.keys() if index in range(ix, ix + col_size)])

            # if there is a meaning in wordnet - add collocation as a new 'word' and delete individual words
            # perhaps too complicated way to do that, but if it's stupid and it works...
            if len(wn.synsets(clean_col)) != 0:
                cols_to_add.append((orig_col, clean_col))
                cols_to_add_ind.append(ix)
                
                to_delete_o2l.append(ix)
                words[ix] = clean_col

                for j in range(ix + col_size - 1, ix, -1):
                    to_delete.append(j)
                    to_delete_o2l.append(j)
                    
        new_indexes = list(range(len(words) - len(to_delete_o2l)))
        new_or_to_lemma = {}
        new_ix = 0
        help_i = 0
        for (k, ix), v in original_to_lemma.items():
            if ix not in to_delete_o2l:
                new_or_to_lemma[(k, new_ix)] = v
                new_ix += 1
            elif ix in cols_to_add_ind:
                new_or_to_lemma[(cols_to_add[help_i][0], new_ix)] = cols_to_add[help_i][1]
                help_i += 1
                new_ix += 1

        original_to_lemma = new_or_to_lemma
        words = [w for (i, w) in enumerate(words) if i not in to_delete]

    original_to_lemma = {k : v for ((k, ix), v) in original_to_lemma.items()}


    return words, original_to_lemma

In [22]:
# create graph for pagerank
# wordnet synsets are vertices
# edge exists between two vertices if there is a wordnet relation (e.g. hypernym/hyponym) between those synsets
def create_graph(clean_words, word_senses):
    G = nx.Graph()
    # create nodes
    for w in clean_words:
        # TODO: try adding pos when getting synsets
        w_synsets = wn.synsets(w)
        if len(w_synsets) > 1:
            for synset in w_synsets:
                G.add_node((w, synset))
        elif len(w_synsets) == 1:
            # if there is only one meaning in wordnet, we choose that one
            word_senses[w] = w_synsets[0]
    #     if there are no synsets - leave None
    #                 G.add_node(synset.name())
    
    # create edges
    for node_i in G.nodes:
        for node_j in G.nodes:
            if node_i[0] != node_j[0] and are_connected(node_i[1], node_j[1]):
                G.add_edge(node_i, node_j)
    
    return G

In [23]:
# determine if two synsets are connected in wordnet
# hypernym/hyponym, holonym/meronym, coordinate (mutual hypernym)
def are_connected(u, v):
    if u in v.hypernyms() or u in v.hyponyms() or u in v.part_holonyms() or u in v.part_meronyms() or u in v.substance_holonyms() or u in v.substance_meronyms():
        return True
#     coordinate relation - e.g. wolf and dog
#     + this will connect two different words that are in the same synset = we think this should be done 
#     it is not specified in the original paper
    u_hyper = set(u.hypernyms())
    v_hyper = set(v.hypernyms())
    if len(u_hyper & v_hyper) > 0:
        return True

In [27]:
# compare the pagerank senses with original senses from semcor file 
def evaluate(pr, word_senses, original_to_lemma, file):
    max_values = dict([(w, 0.0) for w in word_senses.keys()])
    
    # for each word assign sense as the max value from pagerank
    for k, v in pr.items():
        if v > max_values[k[0]]:
            word_senses[k[0]] = k[1]
            max_values[k[0]] = v
    
    # get actual meaning of words from original semcor file
    actual_meaning = get_actual_meaning(file)
    
    # calculate accuracy
    accuracy = calculate_accuracy(original_to_lemma, word_senses, actual_meaning, show_errors=True)
    print('Accuracy: {}'.format(accuracy))
    
    return accuracy

In [28]:
# each word in semcor file has a meaning (wordnet synset) assigned by an expert  
def get_actual_meaning(file):
    actual_meaning = {}
    no_lemma_sense = 0
    trees = list(semcor.tagged_chunks(fileids=file, tag='sem'))
    no_trees = 0
    for tree in trees:
        if isinstance(tree, nltk.tree.Tree):
            actual_meaning['_'.join(tree.leaves()).lower()] = tree.label()
            if not isinstance(tree.label(), nltk.corpus.reader.wordnet.Lemma):
                #TODO: handle these - create Lemma (or Synset) from string if possible...
                # There are mistakes in semcor file: e.g. toe.a.00 sense doesn't exist in wordnet
                no_lemma_sense += 1
        else:
          no_trees += 1
    print('Number of words that do not have lemma sense in semcor file: {}'.format(no_lemma_sense))
    print('Number of words that do not have tree in semcor file: {}'.format(no_trees))
    
    return actual_meaning

In [29]:
# calculate the fraction of words that our algorithm assigned the same sense as in semcor file
def calculate_accuracy(original_to_lemma, word_senses, actual_meaning, show_errors=False):
    correct = 0
    incorrect = 0
    broken_meaning = 0
#     print(actual_meaning)
    error_log = []
    
    for k, v in actual_meaning.items():
        if not isinstance(v, nltk.corpus.reader.wordnet.Lemma):
#             print("Broken meaning in the semcor file for: {}".format(k))
            broken_meaning += 1
        elif k not in original_to_lemma.keys():
            if len(wn.synsets(k)) != 0:
                # they have a collocation that exists in wordnet, but we don't
                # probably contains stopwords...
#                 print("They found a collocation, and we didn't: {}".format(k))
                incorrect += 1
                error_log.append((k, None, v))
        elif original_to_lemma[k] not in word_senses.keys():
            print("This shouldn't happen: {}".format(k))
        elif word_senses[original_to_lemma[k]] is None:
            # probably the word doesn't exist in wordnet...
#             print("They have a meaning, and we don't: {}".format(k))
            incorrect += 1
            error_log.append((k, None, v))
        elif v in word_senses[original_to_lemma[k]].lemmas():
            # we chose the right meaning
            correct += 1
        else:
            # we chose the wrong meaning
            incorrect += 1
            error_log.append((k, word_senses[original_to_lemma[k]], v))


    if show_errors:
        for word, our_sense, their_sense in error_log:
            print('Word: "{}", our sense: {}, their sense: {}'.format(word, our_sense, their_sense))
            
    print("Correct = {}, incorrect = {}".format(correct, incorrect))
    
    return correct / (correct + incorrect)

In [30]:
wn.synsets('jawaharlal_nehru')

[Synset('nehru.n.01')]

In [31]:
wsd()

['brown1/tagfiles/br-g11.xml', 'brown1/tagfiles/br-a02.xml', 'brownv/tagfiles/br-e14.xml', 'brown2/tagfiles/br-p24.xml', 'brown1/tagfiles/br-k28.xml']
Number of words that do not have lemma sense in semcor file: 43
Number of words that do not have tree in semcor file: 1099
Word: "more", our sense: None, their sense: Lemma('more.r.01.more')
Word: "then", our sense: None, their sense: Lemma('then.r.02.then')
Word: "will", our sense: None, their sense: Lemma('will.n.02.will')
Word: "cope_with", our sense: None, their sense: Lemma('meet.v.06.cope_with')
Word: "is", our sense: None, their sense: Lemma('be.v.01.be')
Word: "of_course", our sense: None, their sense: Lemma('naturally.r.01.of_course')
Word: "just", our sense: None, their sense: Lemma('precisely.r.01.just')
Word: "other", our sense: None, their sense: Lemma('other.a.01.other')
Word: "not", our sense: None, their sense: Lemma('not.r.01.not')
Word: "lavoisier", our sense: Synset('lavoisier.n.01'), their sense: Lemma('person.n.01.pe

Number of words that do not have lemma sense in semcor file: 0
Number of words that do not have tree in semcor file: 1946
Word: "are", our sense: None, their sense: Lemma('be.v.05.be')
Word: "rest_on", our sense: None, their sense: Lemma('lean_on.v.01.rest_on')
Word: "have", our sense: None, their sense: Lemma('get.v.03.have')
Word: "be", our sense: None, their sense: Lemma('be.v.01.be')
Word: "does", our sense: None, their sense: Lemma('do.v.04.do')
Word: "having", our sense: None, their sense: Lemma('get.v.03.have')
Word: "is", our sense: None, their sense: Lemma('be.v.01.be')
Word: "has", our sense: None, their sense: Lemma('have.v.02.have')
Word: "do", our sense: None, their sense: Lemma('do.v.04.do')
Word: "put", our sense: Synset('put.v.01'), their sense: Lemma('work.v.12.put_to_work')
Word: "turn_off", our sense: None, their sense: Lemma('turn_off.v.02.turn_off')
Word: "pull_out", our sense: None, their sense: Lemma('pull_out.v.01.pull_out')
Word: "load_up", our sense: None, the

Number of words that do not have lemma sense in semcor file: 36
Number of words that do not have tree in semcor file: 1399
Word: "winston", our sense: None, their sense: Lemma('person.n.01.person')
Word: "better", our sense: Synset('commodity.n.01'), their sense: Lemma('better.r.01.better')
Word: "do", our sense: None, their sense: Lemma('make.v.01.do')
Word: "had", our sense: None, their sense: Lemma('have.v.01.have')
Word: "girl", our sense: None, their sense: Lemma('female_child.n.01.girl')
Word: "were", our sense: None, their sense: Lemma('be.v.05.be')
Word: "was", our sense: None, their sense: Lemma('be.v.01.be')
Word: "just_now", our sense: None, their sense: Lemma('just.r.03.just_now')
Word: "leona", our sense: None, their sense: Lemma('person.n.01.person')
Word: "down", our sense: None, their sense: Lemma('down.r.01.down')
Word: "not", our sense: None, their sense: Lemma('not.r.01.not')
Word: "washed", our sense: Synset('wash.v.04'), their sense: Lemma('washed.s.01.washed')
Wor

In [27]:
# collocations that contain stopwords
wn.synsets('responsible_for')
wn.synsets('out_of_bounds')

[Synset('sideline.n.01')]

In [15]:
wn.synsets('eye-strain')

[]

In [ ]:
# some testing...

In [ ]:
nltk.pos_tag(['refuse'])

[('refuse', 'NN')]

In [ ]:
nltk.pos_tag(['they', 'refuse'])

[('they', 'PRP'), ('refuse', 'VBP')]

In [ ]:
#pos tagging depends on context

In [ ]:
wn.synsets('are') # there are stopwords in wordnet

[Synset('are.n.01'),
 Synset('be.v.01'),
 Synset('be.v.02'),
 Synset('be.v.03'),
 Synset('exist.v.01'),
 Synset('be.v.05'),
 Synset('equal.v.01'),
 Synset('constitute.v.01'),
 Synset('be.v.08'),
 Synset('embody.v.02'),
 Synset('be.v.10'),
 Synset('be.v.11'),
 Synset('be.v.12'),
 Synset('cost.v.01')]

In [ ]:
wn.synsets('bertrand_russell')

[Synset('russell.n.07')]

In [ ]:
wn.synsets('bertrand_russell')[0].lemmas()

[Lemma('russell.n.07.Russell'),
 Lemma('russell.n.07.Bertrand_Russell'),
 Lemma('russell.n.07.Bertrand_Arthur_William_Russell'),
 Lemma('russell.n.07.Earl_Russell')]

In [ ]:
test_words = ['I', 'am', 'Bertrand', 'Arthur', 'William', 'Russell']

In [ ]:
pam = {}

In [ ]:
wn_collocations(test_words, pam)

In [ ]:
test_words

['I', 'am', 'Bertrand_Arthur_William_Russell']

In [ ]:
wn.synsets("n't") # => errors in semcor file ???

[]